In [5]:
import requests
from urllib.request import Request, urlopen
import re
from bs4 import BeautifulSoup

class Webscrapper:
    def __init__(self, mode):
        self.url = ""
        self.mode = mode
        self.soup = ""
    def getMode(self):
        return self.mode 

    def set_Mode(self, mode):
        self.mode = mode

    def getURL(self):
        return self.url 

    def set_URL(self, values):
        #structure for values tuple (0:template for url, 1:tuple of values)
        template = values[0]
        if self.getMode() == "url":
            self.url = template.format(values[1][0], values[1][1])  
            self.url = self.url.replace(" ", "%20")
        if self.getMode() =="job":
            self.url = template.format(values[1])
            print(self.url)

    def createSoup(self):
        req = Request(self.getURL(), headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        self.soup = BeautifulSoup(webpage, 'html.parser')

    def webScrape(self, jobPosts=[], searchQuery=["",""]):
        #URL mode will return a list of  tuples of (jobTitle, jobCompany, JobID)
        cleaner =  re.compile('<.*?>')
        if self.getMode() =="url":
            position = searchQuery[0]
            location = searchQuery[1] 
            jobIDs = []
            self.set_URL(('https://uk.indeed.com/jobs?q={}&l={}',(position, location)))
            self.createSoup()
            cards = self.soup.find_all('div', 'job_seen_beacon')
            mosaic = self.soup.find('div', {"id" : "mosaic-provider-jobcards"})
            jobIDs = self.jobID(mosaic)
            index = 0
            while index < 1:
                c = cards[index]
                tempTitle = self.jobTitle(c)
                title = self.cleanhtml(tempTitle, cleaner)
                tempCompany = self.companyName(c)
                company = self.cleanhtml(tempCompany, cleaner)
                jobPosts.append((title,company,jobIDs[index]))
                index += 1
            return jobPosts
        if self.getMode() == "job":
            jobsList = []
            for j in jobPosts:
                self.set_URL(('https://www.indeed.com/viewjob?viewtype=embedded&jk={}',j[2]))
                self.createSoup()
                jobDesc= self.jobDescription(self.soup)
                clean = self.cleanhtml(jobDesc, cleaner)
                jobsList.append((j[0],j[1],clean))
            return jobsList

    def jobID(self, mosaic):
        mosaicsAnchors = [] 
        ids = []
        pattern = 'result job_\w+'
        tempMosaicsAnchors = self.soup.findAll('a')
        for a in tempMosaicsAnchors:
            if("tapItem fs-unmask result" in str(a)):
                mosaicsAnchors.append(a)
        for m in mosaicsAnchors:
            result = re.findall(pattern, str(m))
            job = result[0].split(' ')[1]
            jobID = job.split('_')[1]
            ids.append(jobID)
        return ids

    def jobTitle(self, card):
        h2 = card.h2
        spans = h2.findAll('span')
        for s in spans:
            if('title' in str(s)):
                jobTitle = s
                return jobTitle
        return None

    #Get Job Company
    def companyName(self, card):  
        spans = card.findAll('span')
        for s in spans:
            if('companyName' in str(s)):
                companyName = s
                return companyName
        return None

    def jobDescription(self, jobSoup):
        return jobSoup.find('div', 'jobsearch-jobDescriptionText')


    def cleanhtml(self,raw_html,cleaned):
            cleantext = re.sub(cleaned, ' ' , str(raw_html))
            return cleantext 

In [6]:
class Job:
    def __init__(self, values):
        self.title = values[0]
        self.company = values[1]
        self.description = values[2]

    def getTitle(self):
        return self.title

    def getCompany(self):
        return self.company

    def getDescription(self):
        return self.description

In [53]:
import pandas as pd
ws = Webscrapper("url")
job = input() 
JobPosts= ws.webScrape(searchQuery=(str(job),''))
ws.set_Mode('job')
JobPostings = ws.webScrape(jobPosts=JobPosts)
#jobs =[Job(list(j)) for j in JobPostings]
jobList = [list(j) for j in JobPostings]
df = pd.DataFrame(jobList, columns = ['Title', 'Company', 'Job Description'])
display(df)
df.to_csv('TestPostings.csv',mode='a', index=False, header=False)
JobPosts.clear()
jobList.clear()

Flight Attendant


IndexError: list index out of range

In [7]:
class Skills:
    def __init__(self, values):
        self.skills = values[0]
        self.softSkills = values[1]

    def getTitle(self):
        return self.skills

    def getCompany(self):
        return self.softSkills

In [8]:
s = Skills(([0],[0]))

In [51]:
s.getCompany()

[0]

In [59]:
extract = pd.read_csv('TestPostings.csv', encoding='ISO-8859-1')

In [60]:
extract

,Title,Company,Job Description
0,Management Accountant,Sofa Club UK,\n Management Accountant \n Purpose of Ro...
1,Business Analyst,MI5,\n \n Business Analyst \n London or Great...
2,"Policy, Programmes and Research Officer",Rethink Mental Illness,\n We have an exciting opportunity for a Poli...
3,Animator,The Acteon Consultancy LLP,\n Want to help shape the moments that matter...
4,Graduate Technical Engineer (Civil / Construc...,Ernest Gordon Recruitment Limited,\n \n Graduate Technical Engineer (Civil / C...
5,School Immunisation Nurse â North Hertfords...,Vaccination UK Ltd,\n Are you a Nurse or Health Professional wit...
6,Freelance Private Chef - UK'S FASTEST GROWING...,Yhangry,\n EARN Â£15K+ P/YEAR FROM 2 SHIFTS A WEEK ...
7,Management Consultant,The Berkeley Partnership,\n When we ask someone to join The Berkeley P...
8,Trainee Solicitor - London (2023),Greenwoods GRM,Our trainee solicitors are key to our firms...
9,Male Security Officer,Heathrow,\n \n Location: Heathrow Airport \n Contr...
